In [ ]:
from dotenv import load_dotenv
import requests
import pprint
import os
import json

load_dotenv()

payload = {
    "email": os.getenv("API_EMAIL"),
    "password": os.getenv("API_PASSWORD")
}

response = requests.post("http://picolo.rothamsted.ac.uk/api/auth/v1/login/", json=payload)

token = ""
headers = None

if response.status_code == 200:
    response_data = response.json()
    print("Login successful!")
    print("Response data:")
    pprint.pp(response_data)
    token = response_data.get("access")
    headers = {"Authorization": f"Bearer {token}"}
else:
    print("Login failed!")
    print("Status code:", response.status_code)
    print("Response:", response.text)

In [ ]:
def count_all_properties(json_data):
    count = 0

    if isinstance(json_data, dict):
        # For each key in the dictionary, increment the count
        count += len(json_data)
        # Recursively count properties in each value
        for value in json_data.values():
            count += count_all_properties(value)
    
    elif isinstance(json_data, list):
        # Recursively count properties in each item of the list
        for item in json_data:
            count += count_all_properties(item)

    return count

In [ ]:
def display_result(protected_response,withJson):
    try:
        json_data = protected_response.json()

        if protected_response.status_code == 200:
            print("Accessed protected endpoint successfully!")
            print(count_all_properties(json_data))
            if withJson:
                pprint.pp(json_data)
        else:
            print("Failed to access protected endpoint.")
            print("Status code: ", protected_response.status_code)
            print("Error message: ",json_data["error"]) 
    except json.JSONDecodeError as e:
        print("error")
        print(f"Error: Malformed JSON data. Details: {e.msg} at line {e.lineno} column {e.colno}")
        print("Response content:", protected_response.text)  # Show the actual response for inspection
        return 0

In [ ]:
api_url = "http://picolo.rothamsted.ac.uk/"

In [ ]:
protected_url = api_url + "api/events/v2/getFieldEvents/?catchments=catchment 4, catchment 6&from_date=2015-6-12&to_date=2015-10-12"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)


In [ ]:
protected_url = api_url + "api/animals/v1/getAnimalCount/?date_start=2011-01-01&date_end=2020-12-31"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/animals/v1/getAnimalInputs/"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/animals/v1/getBasicAnimalData/?start_grazingyear=2010&end_grazingyear=2020&farmlet_name=Red"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/animals/v1/getGrazingQuality/"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/animals/v1/getGrazingQuality/?from_date=2011-6-12&to_date=2018-10-12"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/animals/v1/getGreenFeed"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/animals/v1/getGreenFeed?start_date=2011-6-12&end_date=2024-10-12"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/animals/v1/getLivestockPerformance?animalId=100"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,True)

In [ ]:
protected_url = api_url + "api/animals/v1/getLivestockPerformanceSales?animalId=100"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,True)

In [ ]:
protected_url = api_url + "api/animals/v1/getSurveyMeasurements?from_date=2018-6-12&to_date=2018-10-12"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,True)

In [ ]:
protected_url = api_url + "api/catchments/v1/getCatchments/"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/catchments/v1/getMeasurementTypes/"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/events/v2/getFieldEvents/"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/events/v2/getFieldEvents/?catchments=Catchment 1, Catchment 3"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [ ]:
protected_url = api_url + "api/events/v2/getFieldEvents/?catchments=Catchment 1, Catchment 3&from_date=2018-06-01"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,True)

In [ ]:
protected_url = api_url + "api/fields/v1/getFieldevents/"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

In [29]:
protected_url = api_url + "api/fields/v1/getFieldevents/?startDate=2018-06-01"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

Accessed protected endpoint successfully!
3578


In [30]:
protected_url = api_url + "api/fields/v1/getFields/"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,False)

Accessed protected endpoint successfully!
370


In [31]:
protected_url = api_url + "api/measurements/v2/getMeasurementsByTypeID/"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,True)

Failed to access protected endpoint.
Status code:  400
Error message:  Add a 'from_date' and we will display a period of 3 months.


In [32]:
protected_url = api_url + "api/measurements/v2/getMeasurementsByTypeID/?from_date=2018-06-01"
protected_response = requests.get(protected_url, headers=headers)
display_result(protected_response,True)

error
Error: Malformed JSON data. Details: Expecting value at line 1 column 1
Response content: <!DOCTYPE html>
<html lang="en">
<head>
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <meta name="robots" content="NONE,NOARCHIVE">
  <title>AttributeError
          at /api/measurements/v2/getMeasurementsByTypeID/</title>
  <style type="text/css">
    html * { padding:0; margin:0; }
    body * { padding:10px 20px; }
    body * * { padding:0; }
    body { font:small sans-serif; background-color:#fff; color:#000; }
    body>div { border-bottom:1px solid #ddd; }
    h1 { font-weight:normal; }
    h2 { margin-bottom:.8em; }
    h3 { margin:1em 0 .5em 0; }
    h4 { margin:0 0 .5em 0; font-weight: normal; }
    code, pre { font-size: 100%; white-space: pre-wrap; word-break: break-word; }
    summary { cursor: pointer; }
    table { border:1px solid #ccc; border-collapse: collapse; width:100%; background:white; }
    tbody td, tbody th { vertical-align:top; padding:2px 3p

0